In [2]:
suppressPackageStartupMessages(library(readxl))
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(patchwork))
suppressPackageStartupMessages(library(pheatmap))
suppressPackageStartupMessages(library(stringr))
library(hise)
library(plyr)
library(purrr)
library(vegan)
#library(arrow)
library(rstatix)
library(parallel)
library(ggpubr)
library(ComplexHeatmap)
library(compositions)
library(ggsci)
library(ggpubr)

ERROR: Error in library(compositions): there is no package called ‘compositions’


In [3]:
colors = c('#FDC086','#FFFF99','#F0027F','#BF5B17','#666666',
          '#1B9E77','#D95F02','#7570B3','#E7298A','#66A61E','#E6AB02','#A6761D','#666666',
          '#A6CEE3','#1F78B4','#B2DF8A','#33A02C','#FB9A99','#E31A1C','#FDBF6F','#FF7F00',
          '#CAB2D6','#6A3D9A','#FFFF99','#B15928','#FBB4AE','#B3CDE3','#CCEBC5','#DECBE4',
          '#FED9A6','#FFFFCC','#E5D8BD','#FDDAEC','#F2F2F2','#B3E2CD','#FDCDAC','#CBD5E8',
          '#F4CAE4','#E6F5C9','#FFF2AE','#F1E2CC','#CCCCCC','#E41A1C','#377EB8','#4DAF4A',
          '#984EA3','#FF7F00','#FFFF33','#A65628','#F781BF','#999999','#66C2A5','#FC8D62',
          '#8DA0CB','#E78AC3','#A6D854','#FFD92F','#E5C494','#B3B3B3','#8DD3C7','#FFFFB3',
          '#BEBADA','#FB8072','#80B1D3','#FDB462','#B3DE69','#FCCDE5','#D9D9D9','#BC80BD',
          '#CCEBC5','#FFED6F')


# Reading the file

In [4]:
# Read the CSV file into a data frame
all_df <- read.csv('/home/jupyter/Myeloid_cells/files/Parquet_files/AIFI_Reference_processed.csv')


In [5]:
#remove UPI from the cohort
all_df <- all_df %>% filter(cohort.cohortGuid != 'UP1')

In [7]:
all_df <- all_df %>%
  filter(!(AIFI_L1 %in% c('ILC', 'Erythrocyte', 'Platelet')))

In [8]:
#colnames(myeloid_df)

In [9]:
rownames(all_df)<-all_df$cell_uuid

In [11]:
unique(all_df$cohort.cohortGuid)

[1] "BR1" "BR2"

# CLR Transform


In [12]:
clr_transform <- function(x) {
  if (length(x) == 0) {
    return(NA)  # return NA for empty vectors
  }
  geom_mean <- exp(mean(log(x)))
  return(log(x / geom_mean))
}

In [ ]:
#example code
columns = ['temperature','humidity' ,'pressure','rain' ,'lightavgw/o0','lightmax','moisture']
for col in columns:
    plt.figure()   # plots figure for each iteration
    plt.boxplot(df[col])

# Comparsion within total PBMCs (Percentage and Percentage CLR)

In [ ]:
## Percentage of total

result <- all_df %>%
  dplyr::group_by(AIFI_L3, subject.subjectGuid) %>%               
  dplyr::summarise(count = n()) %>%                  # Count occurrences
  dplyr::group_by(subject.subjectGuid) %>%                       
  dplyr::mutate(percentage = (count / sum(count)) * 100) %>% 
  dplyr::group_by(subject.subjectGuid) %>%     
  dplyr::mutate(percentage_clr = clr_transform((percentage))) %>% # Group again by subjectGuid for CLR# Calculate percentage
  dplyr::ungroup()

head(result)
#sum(result$subject.subjectGuid)

In [ ]:
result %>% filter(subject.subjectGuid == "BR1001") %>% 
      select(percentage) %>%
      pull() %>% 
      sum()

In [ ]:
result<-left_join(result,unique(all_df[c('cohort.cohortGuid','subject.subjectGuid','subject.biologicalSex','CMV.IgG.Serology.Result.Interpretation')]),by=c('subject.subjectGuid'))

In [ ]:
head(result)


In [ ]:
table(result$AIFI_L3)

# Specific cell types (Parent)

In [ ]:
cell = "T cell"

In [ ]:
cell_df <- all_df %>% dplyr::filter(AIFI_L1 == cell)


In [ ]:
head(cell_df)

In [ ]:
result_final <- cell_df %>%
  dplyr::group_by(AIFI_L3, subject.subjectGuid) %>%               
  dplyr::summarise(count = n()) %>%                  # Count occurrences
  dplyr::group_by(subject.subjectGuid) %>%                       
  dplyr::mutate(percentage = (count / sum(count)) * 100) %>% 
  dplyr::group_by(subject.subjectGuid) %>%     
  dplyr::mutate(percentage_clr = clr_transform((percentage))) %>% # Group again by subjectGuid for CLR# Calculate percentage
  dplyr::ungroup()




In [ ]:
head(result_final)

In [ ]:
result_final<-left_join(result_final,unique(all_df[c('cohort.cohortGuid','subject.subjectGuid','subject.biologicalSex','CMV.IgG.Serology.Result.Interpretation')]),by=c('subject.subjectGuid'))

In [ ]:
result_final$facet_combined <- paste(result_final$cohort.cohortGuid, result_final$CMV.IgG.Serology.Result.Interpretation, sep = "_")
result_final$facet_combined<- factor(result_final$facet_combined)

In [ ]:
head(result_final)

In [ ]:
#New loop

In [ ]:
cell_L3 = "CD4 MAIT"

In [ ]:
L3_df <- result_final %>% dplyr::filter(AIFI_L3 == "CD4 MAIT")

In [ ]:
head(L3_df)

In [ ]:
stat_test <- wilcox_test(percentage ~ facet_combined, data = L3_df, comparisons = my_comparisons, paired= FALSE)%>% 
       adjust_pvalue(method = 'BH') %>% 
       add_xy_position(x = "facet_combined", dodge = 0.8)

In [ ]:
head(stat_test)

In [ ]:
 outfile <- paste("/home/jupyter/Myeloid_cells/Plots/CLR_Comparisons_cohort_&_CMV_Parent/CLR_Comparisons_cohort_&_CMV_parent_TEST", cell_L3, ".pdf", sep="")
    options(repr.plot.width = 18, repr.plot.height = 10)
    p3 <- ggplot(L3_df, aes(x = facet_combined, y = percentage_clr, color = CMV.IgG.Serology.Result.Interpretation)) +
      geom_boxplot(width = 0.5, outlier.shape = NA) +
      geom_jitter(width =0.1) +
      ylab('percentage_clr') + 
      xlab(cell) +
      scale_color_manual(values = c("green", "red", "orange", "blue")) +
      theme(axis.text.x = element_text(size = 12, angle = 90 ),     # Adjust font size of x-axis text
            axis.text.y = element_text(size = 12),     # Adjust font size of y-axis text
            axis.title = element_text(size = 14),
            panel.background = element_rect(fill = "white"),
            axis.line = element_line(color = "black"),  # Set axis line color to black,  # Set major grid line color to black
            panel.grid.minor = element_blank(),
            legend.position = "top",  # Set legend position to top
            #legend.title = element_blank()
           ) +
      geom_boxplot(position = position_dodge(width = 1))+
      stat_pvalue_manual(stat_test, label = "p.adj.signif", tip.length = 0) +
      scale_y_continuous(expand = expansion(mult = c(0, 0.1)))
      pdf(file = outfile, width = 7, height = 11)
      print(p3)
      dev.off()



# Final Loop code

In [14]:
unique_AIFI_L1 <- unique(all_df$AIFI_L1)
unique_AIFI_L1
#unique_AIFI_L3 <- unique(result_final$AIFI_L3)
#unique_AIFI_L3

[1] "T cell"          "NK cell"         "Monocyte"        "DC"             
[5] "Progenitor cell" "B cell"

In [15]:
##For CLR - PARENT PLOT

my_comparisons <- list(c("BR1_Negative", "BR2_Negative"), c("BR1_Negative", "BR1_Positive"), c("BR1_Positive", "BR2_Positive"), c("BR2_Negative", "BR2_Positive") )

for(cell in unique_AIFI_L1){
    
    print(cell)
    
    #rm(cell_df, result_final)
    
    cell_df <- all_df %>% dplyr::filter(AIFI_L1 == cell)

    
    result_final <- cell_df %>%
      dplyr::group_by(AIFI_L3, subject.subjectGuid) %>%               
      dplyr::summarise(count = n()) %>%                  # Count occurrences
      dplyr::group_by(subject.subjectGuid) %>%                       
      dplyr::mutate(percentage = (count / sum(count)) * 100) %>% 
      dplyr::group_by(subject.subjectGuid) %>%     
      dplyr::mutate(percentage_clr = clr_transform((percentage))) %>% # Group again by subjectGuid for CLR# Calculate percentage
      dplyr::ungroup()

    
    result_final<-left_join(result_final,unique(all_df[c('cohort.cohortGuid','subject.subjectGuid','subject.biologicalSex','CMV.IgG.Serology.Result.Interpretation')]),by=c('subject.subjectGuid'))
    
    result_final$facet_combined <- paste(result_final$cohort.cohortGuid, result_final$CMV.IgG.Serology.Result.Interpretation, sep = "_")
    result_final$facet_combined<- factor(result_final$facet_combined)
    
    for(cell_L3 in unique(result_final$AIFI_L3)){
        
        #rm(L3_df)
        
        L3_df <- result_final %>% dplyr::filter(AIFI_L3 == cell_L3)
        
        stat_test <- wilcox_test(percentage_clr ~ facet_combined, data = L3_df, comparisons = my_comparisons, paired= FALSE)%>% 
               adjust_pvalue(method = 'BH') %>% 
               add_xy_position(x = "facet_combined", dodge = 0.8)




        outfile <- paste("/home/jupyter/Myeloid_cells/Plots/CLR_Comparisons_cohort_&_CMV_Parent/CLR_Comparisons_cohort_&_CMV_parent_", cell_L3, ".pdf", sep="")
        options(repr.plot.width = 18, repr.plot.height = 10)
        p3 <- ggplot(L3_df, aes(x = facet_combined, y = percentage_clr, color = CMV.IgG.Serology.Result.Interpretation)) +
          geom_boxplot(width = 0.5, outlier.shape = NA) +
          geom_jitter(width =0.1) +
          ylab('percentage_clr') + 
          xlab(cell_L3) +
          scale_color_manual(values = c("green", "red", "orange", "blue")) +
          theme(axis.text.x = element_text(size = 12, angle = 90 ),     # Adjust font size of x-axis text
                axis.text.y = element_text(size = 12),     # Adjust font size of y-axis text
                axis.title = element_text(size = 14),
                panel.background = element_rect(fill = "white"),
                axis.line = element_line(color = "black"),  # Set axis line color to black,  # Set major grid line color to black
                panel.grid.minor = element_blank(),
                legend.position = "top",  # Set legend position to top
                #legend.title = element_blank()
               ) +
          geom_boxplot(position = position_dodge(width = 1))+
          stat_pvalue_manual(stat_test, label = "p.adj.signif", tip.length = 0) +
          scale_y_continuous(expand = expansion(mult = c(0, 0.1)))
          pdf(file = outfile, width = 7, height = 11)
          print(p3)
          dev.off()

        
        }
    
    
    }





[1] "T cell"


`summarise()` has grouped output by 'AIFI_L3'. You can override using the
`.groups` argument.


[1] "NK cell"


`summarise()` has grouped output by 'AIFI_L3'. You can override using the
`.groups` argument.


[1] "Monocyte"


...

`summarise()` has grouped output by 'AIFI_L3'. You can override using the
`.groups` argument.


[1] "Progenitor cell"


`summarise()` has grouped output by 'AIFI_L3'. You can override using the
`.groups` argument.


[1] "B cell"


`summarise()` has grouped output by 'AIFI_L3'. You can override using the
`.groups` argument.


In [16]:
##For Percentage - PARENT PLOT

my_comparisons <- list(c("BR1_Negative", "BR2_Negative"), c("BR1_Negative", "BR1_Positive"), c("BR1_Positive", "BR2_Positive"), c("BR2_Negative", "BR2_Positive") )

for(cell in unique_AIFI_L1){
    
    print(cell)
    
    #rm(cell_df, result_final)
    
    cell_df <- all_df %>% dplyr::filter(AIFI_L1 == cell)

    
    result_final <- cell_df %>%
      dplyr::group_by(AIFI_L3, subject.subjectGuid) %>%               
      dplyr::summarise(count = n()) %>%                  # Count occurrences
      dplyr::group_by(subject.subjectGuid) %>%                       
      dplyr::mutate(percentage = (count / sum(count)) * 100) %>% 
      dplyr::group_by(subject.subjectGuid) %>%     
      dplyr::mutate(percentage_clr = clr_transform((percentage))) %>% # Group again by subjectGuid for CLR# Calculate percentage
      dplyr::ungroup()

    
    result_final<-left_join(result_final,unique(all_df[c('cohort.cohortGuid','subject.subjectGuid','subject.biologicalSex','CMV.IgG.Serology.Result.Interpretation')]),by=c('subject.subjectGuid'))
    
    result_final$facet_combined <- paste(result_final$cohort.cohortGuid, result_final$CMV.IgG.Serology.Result.Interpretation, sep = "_")
    result_final$facet_combined<- factor(result_final$facet_combined)
    
    for(cell_L3 in unique(result_final$AIFI_L3)){
        
        #rm(L3_df)
        
        L3_df <- result_final %>% dplyr::filter(AIFI_L3 == cell_L3)
        
        stat_test <- wilcox_test(percentage ~ facet_combined, data = L3_df, comparisons = my_comparisons, paired= FALSE)%>% 
               adjust_pvalue(method = 'BH') %>% 
               add_xy_position(x = "facet_combined", dodge = 0.8)




        outfile <- paste("/home/jupyter/Myeloid_cells/Plots/Percentage_Comparisons_cohort_&_CMV_Parent/Percentage_Comparisons_cohort_&_CMV_parent_", cell_L3, ".pdf", sep="")
        options(repr.plot.width = 18, repr.plot.height = 10)
        p3 <- ggplot(L3_df, aes(x = facet_combined, y = percentage, color = CMV.IgG.Serology.Result.Interpretation)) +
          geom_boxplot(width = 0.5, outlier.shape = NA) +
          geom_jitter(width =0.1) +
          ylab('percentage') + 
          xlab(cell_L3) +
          scale_color_manual(values = c("green", "red", "orange", "blue")) +
          theme(axis.text.x = element_text(size = 12, angle = 90 ),     # Adjust font size of x-axis text
                axis.text.y = element_text(size = 12),     # Adjust font size of y-axis text
                axis.title = element_text(size = 14),
                panel.background = element_rect(fill = "white"),
                axis.line = element_line(color = "black"),  # Set axis line color to black,  # Set major grid line color to black
                panel.grid.minor = element_blank(),
                legend.position = "top",  # Set legend position to top
                #legend.title = element_blank()
               ) +
          geom_boxplot(position = position_dodge(width = 1))+
          stat_pvalue_manual(stat_test, label = "p.adj.signif", tip.length = 0) +
          scale_y_continuous(expand = expansion(mult = c(0, 0.1)))
          pdf(file = outfile, width = 7, height = 11)
          print(p3)
          dev.off()

        
        }
    
    
    }





[1] "T cell"


`summarise()` has grouped output by 'AIFI_L3'. You can override using the
`.groups` argument.


[1] "NK cell"


`summarise()` has grouped output by 'AIFI_L3'. You can override using the
`.groups` argument.


[1] "Monocyte"


...

`summarise()` has grouped output by 'AIFI_L3'. You can override using the
`.groups` argument.


[1] "Progenitor cell"


`summarise()` has grouped output by 'AIFI_L3'. You can override using the
`.groups` argument.


[1] "B cell"


`summarise()` has grouped output by 'AIFI_L3'. You can override using the
`.groups` argument.
